# Lab 6 - Threat Hunting with Data Science
**Security Data Science**

Sergio Marchena - 16387

UVG - 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import random

pd.set_option("display.max_columns", None)

## Parte 1: Filtrado y preprocesamiento 

In [2]:
data = [json.loads(line) for line in open('large_eve.json', 'r')]

### 1. Cargue  la  información  del  archivo  large_even.json  en  una  lista,  muestre  la  cantidad  de registros total (deben ser 746, 909).

In [3]:
print('Hay %d registros.' %len(data))

Hay 746909 registros.


### 2. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos  interesados  en  los  registros  DNS.  Cargue  únicamente  aquellos  registros  en  cuya  llave  se encuentra “DNS”. 

In [4]:
dns = [d for d in data if d['event_type'] == 'dns']

• Muestre la nueva cantidad de registros filtrados. 

In [5]:
print('Hay %d registros DNS.'%len(dns))

Hay 15749 registros DNS.


• Muestre la información de 2 registros cualesquiera. 

In [6]:
random.sample(dns, 2)

[{'timestamp': '2017-07-22T19:05:17.375505-0500',
  'flow_id': 36245400828625,
  'pcap_cnt': 2358599,
  'event_type': 'dns',
  'vlan': 110,
  'src_ip': '192.168.201.72',
  'src_port': 42548,
  'dest_ip': '192.168.207.4',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 63112,
   'rrname': 'safebrowsing.clients.google.com',
   'rrtype': 'AAAA',
   'tx_id': 0}},
 {'timestamp': '2017-07-22T18:29:18.228037-0500',
  'flow_id': 24077616970437,
  'pcap_cnt': 506138,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.205.188',
  'src_port': 53356,
  'dest_ip': '192.168.207.4',
  'dest_port': 53,
  'proto': 'UDP',
  'dns': {'type': 'query',
   'id': 30659,
   'rrname': 'centos.mirrors.tds.net',
   'rrtype': 'AAAA',
   'tx_id': 0}}]

• Debido a que la data consiste en json anidados, utilice la característica json_normalize para 
normalizar la información y asignarla en un dataframe. 

In [7]:
df = pd.json_normalize(dns)
df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15744,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,en-us.start3.mozilla.com,NaN,NaN,NXDOMAIN,NaN,NaN
15745,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
15746,2017-07-22T19:47:38.146503-0500,2199315651640391,4452917,dns,130,192.168.203.67,50975,192.168.207.4,53,UDP,query,35892,whitecell.localdomain,AAAA,0.0,NaN,NaN,NaN
15747,2017-07-22T19:47:38.146953-0500,2199315651640391,4452918,dns,130,192.168.207.4,53,192.168.203.67,50975,UDP,answer,35892,whitecell.localdomain,NaN,NaN,NXDOMAIN,NaN,NaN


• Como  estamos  buscando  dominios  DGA,  debemos  filtrar  los  registros  DNS  para  aquellos 
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió 
bajar la cantidad a 2849 registros. 

In [8]:
dnsA = df[df['dns.rrtype'] == 'A']
print('Hay %d registros DNS.'%len(dnsA))

Hay 2849 registros DNS.


• Filtre los dominios únicos. 

In [9]:
unique_domains = dnsA.drop_duplicates(subset=['dns.rrname'])
unique_domains

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15632,2017-07-22T19:44:29.056834-0500,787856241843714,4405786,dns,130,192.168.203.69,60088,192.168.207.4,53,UDP,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN
15661,2017-07-22T19:45:10.437871-0500,1612464195546735,4417023,dns,160,192.168.207.4,11162,192.168.206.44,53,UDP,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN
15701,2017-07-22T19:47:38.301392-0500,515168780589392,4452953,dns,120,192.168.202.172,51325,192.168.207.4,53,UDP,query,54310,ocsp.verisign.com,A,0.0,NaN,NaN,NaN
15716,2017-07-22T19:39:02.426497-0500,572502265201153,4319593,dns,180,192.168.198.58,42157,192.168.207.4,53,UDP,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN


In [10]:
domains = unique_domains['dns.rrname'].value_counts()
domains

api.wunderground.com                      1
www.cakephp.org                           1
mirror.sanctuaryhost.com                  1
mirror.team-cymru.org                     1
mirror.umoss.org                          1
                                         ..
sourceforge.net                           1
www.freepbx.org                           1
secure.informaction.com.hackerlabs.vpn    1
www.bigflickrfeed.com                     1
client-software.real.com                  1
Name: dns.rrname, Length: 177, dtype: int64

• Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare 
como una columna nueva. 

In [11]:
from flare.data_science.features import domain_tld_extract

In [13]:
# Domain TLD Extract
unique_domains['TLD_name'] = unique_domains['dns.rrname'].apply(domain_tld_extract)
unique_domains.sample(10)

/var/folders/3q/f_c12djx2yxc8v2vq4255gcm0000gn/T/ipykernel_40668/629738443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_domains['TLD_name'] = unique_domains['dns.rrname'].apply(domain_tld_extract)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,TLD_name
554,2017-07-22T17:37:20.466444-0500,1258201905372684,75806,dns,150,192.168.205.170,48918,192.168.207.4,53,UDP,query,38493,AOLDTCMA04.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
175,2017-07-22T17:36:28.423097-0500,1718188604421305,70787,dns,110,192.168.201.68,55126,192.168.207.4,53,UDP,query,10517,secure.informaction.com,A,0.0,NaN,NaN,NaN,informaction.com
2988,2017-07-22T18:29:18.784373-0500,2173783910578165,506977,dns,150,192.168.205.188,47202,192.168.207.4,53,UDP,query,55297,mirror.rocketinternet.net,A,0.0,NaN,NaN,NaN,rocketinternet.net
10048,2017-07-22T19:22:18.553308-0500,836902533624156,2942241,dns,180,192.168.198.60,33725,192.168.207.4,53,UDP,query,40284,1922.168.22.254.home,A,0.0,NaN,NaN,NaN,1922.168.22.254.home
120,2017-07-22T17:37:20.468186-0500,98401821664474,75808,dns,150,192.168.205.170,30692,192.168.207.4,53,UDP,query,48331,AOLDTCMA04.ad.aol.aoltw.net.office.aol.com,A,0.0,NaN,NaN,NaN,aol.com
2132,2017-07-22T18:29:17.253025-0500,1479442317565025,504317,dns,150,192.168.205.188,58069,192.168.207.4,53,UDP,query,27829,centos.cs.wisc.edu,A,0.0,NaN,NaN,NaN,wisc.edu
2550,2017-07-22T18:29:17.985020-0500,937950020700092,505834,dns,150,192.168.205.188,55467,192.168.207.4,53,UDP,query,6453,mirrors.xmission.com,A,0.0,NaN,NaN,NaN,xmission.com
7918,2017-07-22T18:42:45.515953-0500,261748275142513,1817249,dns,110,192.168.201.57,63446,192.168.207.4,53,UDP,query,42400,aosnotify.me.com,A,0.0,NaN,NaN,NaN,me.com
1075,2017-07-22T17:55:44.595424-0500,1427565423105504,243121,dns,150,192.168.205.188,58076,192.168.207.4,53,UDP,query,16039,www.google.com,A,0.0,NaN,NaN,NaN,google.com
800,2017-07-22T17:41:17.679111-0500,2178012117032135,99878,dns,140,192.168.204.59,43901,192.168.207.4,53,UDP,query,60867,secure.informaction.com.stayonline.net,A,0.0,NaN,NaN,NaN,stayonline.net
